In [42]:
import numpy as np  # type: ignore
import matplotlib.pyplot as plt  # type: ignore
import pandas as pd  # type: ignore

In [43]:
NUM_ITERATIONS = 10
NUM_WOLFS = 10

In [44]:
NUM_INTERVALS = 96

In [45]:
# Matriz de carros: 1-Modelo | 2-Capacidade (kM) | 3-ID | 4-disponibilidade | 5-viagem | 6-Capacidade | 7-Capacidade da bateria (kWh) | 8-Taxa de carga da bateria (kW) | 9-Taxa de descarga da bateria (kW)
cars_array = np.array(
    [
        [1, 170, 1, 0, 0, 4, 44.9, 0.2 * 44.9, -50 / 4, (60 / 170) * 44.9 * 0.25],
        [2, 230, 2, 0, 0, 5, 229, 0.2 * 229, -100 / 4, (60 / 230) * 229 * 0.25],
        [3, 230, 3, 0, 0, 3, 174, 0.2 * 174, -100 / 4, (60 / 230) * 174 * 0.25],
    ]
)
df = pd.DataFrame(
    cars_array,
    columns=[
        "Modelo",
        "Capacidade (km)",
        "ID",
        "Disponibilidade",
        "Viagem",
        "Capacidade",
        "Capacidade da bateria (kWh)",
        "Taxa de carga da bateria (kW)",
        "Taxa de descarga da bateria (kW)",
        "Taxa de carga específica (kW)",
    ],
)

df.head()

,Modelo,Capacidade (km),ID,Disponibilidade,Viagem,Capacidade,Capacidade da bateria (kWh),Taxa de carga da bateria (kW),Taxa de descarga da bateria (kW),Taxa de carga específica (kW)
0,1.0,170.0,1.0,0.0,0.0,4.0,44.9,8.98,-12.5,3.961765
1,2.0,230.0,2.0,0.0,0.0,5.0,229.0,45.80,-25.0,14.934783
2,3.0,230.0,3.0,0.0,0.0,3.0,174.0,34.80,-25.0,11.347826


In [46]:
cars = [
    {
        "autonomy_km": car[1],
        "autonomy_kwh": car[6],
        "tx_charge": car[7],
        "tx_discharge": np.abs(car[8]),
    }
    for car in cars_array
]

In [47]:
for car in cars:
    print(car)

{'autonomy_km': np.float64(170.0), 'autonomy_kwh': np.float64(44.9), 'tx_charge': np.float64(8.98), 'tx_discharge': np.float64(12.5)}
{'autonomy_km': np.float64(230.0), 'autonomy_kwh': np.float64(229.0), 'tx_charge': np.float64(45.800000000000004), 'tx_discharge': np.float64(25.0)}
{'autonomy_km': np.float64(230.0), 'autonomy_kwh': np.float64(174.0), 'tx_charge': np.float64(34.800000000000004), 'tx_discharge': np.float64(25.0)}


In [48]:
travels = [
    ((15, 20), (48, 55), (90, 92)),
    ((17, 24), (90, 92)),
    ((50, 55), (76, 89)),
]

In [49]:
def is_travelling_in_time(travels):
    is_travelling = np.zeros(NUM_INTERVALS, dtype=int)

    for start, end in travels:
        is_travelling[start:end] = 1

    is_travelling = np.array(is_travelling)

    return is_travelling

In [50]:
is_cars_travelling = [is_travelling_in_time(travel) for travel in travels]
for is_travelling in is_cars_travelling:
    print(*is_travelling)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0


In [51]:
def calculate_time_to_charge_for_each_travel(travels, tx_charge, tx_discharge):
    spent_energy = [(end - start) * tx_discharge for start, end in travels]
    time_to_charge = np.ceil(spent_energy / tx_charge).astype(int)
    return time_to_charge

In [54]:
time_to_charge_for_travels = [
    [
        calculate_time_to_charge_for_each_travel(
            travel, car["tx_charge"], car["tx_discharge"]
        )
        for travel in travels
    ]
    for car in cars
]
time_to_charge_for_travels

[[array([ 7, 10,  3]), array([10,  3]), array([ 7, 19])],
 [array([3, 4, 2]), array([4, 2]), array([3, 8])],
 [array([4, 6, 2]), array([6, 2]), array([ 4, 10])]]

In [55]:
def find_empty_intervals(occupied_intervals):
    empty_intervals = []
    empty_start = 0

    for start, end in occupied_intervals:
        if empty_start < start:
            empty_intervals.append((empty_start, start - 1))
        empty_start = end + 1

    if empty_start < NUM_INTERVALS:
        empty_intervals.append((empty_start, NUM_INTERVALS - 1))

    return empty_intervals

In [56]:
cars_available_intervals = [find_empty_intervals(travel) for travel in travels]
cars_available_intervals

[[(0, 14), (21, 47), (56, 89), (93, 95)],
 [(0, 16), (25, 89), (93, 95)],
 [(0, 49), (56, 75), (90, 95)]]